# Minimap Generate Tryout



In [1]:
import os
import sys
import shutil
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from pathlib import Path
import random
from tqdm import tqdm

sys.path.append('./')
from minimap_generator import ImageDrawer

print("Importing ImageDrawer")


Importing ImageDrawer


## 1. Setup


In [2]:
# motion blur + gaussian blur helper functions
import random
import cv2
import numpy as np

def motion_blur(img, k=3, angle=15):
    # build horizontal kernel and rotate
    kernel = np.zeros((k, k), dtype=np.float32)
    kernel[k // 2, :] = 1.0
    M = cv2.getRotationMatrix2D((k / 2 - 0.5, k / 2 - 0.5), angle, 1.0)
    kernel = cv2.warpAffine(kernel, M, (k, k))
    s = kernel.sum()
    if s > 0:
        kernel /= s
    return cv2.filter2D(img, -1, kernel)

def blur_icons(img_uint8, annotations, prob=0.3):
    """
    Apply motion blur + gaussian blur to the icon area of YOLO annotations with probability prob.
    img_uint8: HWC, uint8
    annotations: ['class cx cy w h'] list (normalized coordinates)
    """
    if random.random() >= prob:
        return img_uint8
    h, w = img_uint8.shape[:2]
    out = img_uint8.copy()
    for ann in annotations:
        parts = ann.strip().split()
        if len(parts) < 5:
            continue
        _, cx, cy, bw, bh = parts
        cx, cy, bw, bh = map(float, (cx, cy, bw, bh))
        x1 = int((cx - bw / 2) * w); y1 = int((cy - bh / 2) * h)
        x2 = int((cx + bw / 2) * w); y2 = int((cy + bh / 2) * h)
        x1 = max(0, x1); y1 = max(0, y1); x2 = min(w, x2); y2 = min(h, y2)
        if x2 <= x1 or y2 <= y1:
            continue
        patch = out[y1:y2, x1:x2]

        k_mb = 3
        angle = 15
        patch = motion_blur(patch, k=k_mb, angle=angle)

        k_g = 3
        if k_g % 2 == 0:
            k_g += 1
        patch = cv2.GaussianBlur(patch, (k_g, k_g), sigmaX=1)
        out[y1:y2, x1:x2] = patch
    return out

print("motion blur + gaussian blur helper functions loaded")


motion blur + gaussian blur helper functions loaded


In [11]:
# setup assets paths
champion_circle_icon_path = "./minimap_sources/champion_icons_circular_small"
minimap_path = "./minimap_sources/minimap_assets"
fog_path = "./minimap_sources/fog_assets"
misc_path = "./minimap_sources/misc_assets"

# output image size
resize = (640, 640)  # YOLO common size

# dataset parameters
minimap_samples = 1  # minimap sample number
output_base_dir = "./generated_minimap_sample"  # dataset root directory

print("paths setup completed")
print(f"minimap sample number: {minimap_samples}")
print(f"output image size: {resize}")


paths setup completed
minimap sample number: 1
output image size: (640, 640)


## 2. Initialize ImageDrawer


In [12]:
try:
    drawer = ImageDrawer(
        champion_circle_icon_path=champion_circle_icon_path,
        minimap_path=minimap_path,
        fog_path=fog_path,
        misc_path=misc_path,
        resize=resize
    )
    print("ImageDrawer initialized successfully")
    print(f"loaded {len(drawer.champion_icons)} champion icons")
    print(f"loaded {len(drawer.minimaps)} minimap backgrounds")
    print(f"loaded {len(drawer.fogs)} fog images")
    print(f"loaded {len(drawer.miscs)} other assets")
except Exception as e:
    print(f"initialization failed: {e}")
    import traceback
    traceback.print_exc()


ImageDrawer initialized successfully
loaded 172 champion icons
loaded 18 minimap backgrounds
loaded 10 fog images
loaded 263 other assets


## 3. Generate Dataset


In [13]:
# generate training set (with 30% probability of icon motion blur + gaussian blur)
print("generating training set...")
for i in tqdm(range(minimap_samples), desc="generating training set"):
    try:
        final_image, yolo_data = drawer.generate_data(i, allow_overlap=False)
        
        # save image
        img_np = final_image.numpy()
        if img_np.shape[0] == 3:  # CHW format
            img_np = img_np.transpose(1, 2, 0)  # convert to HWC format
        img_np = np.clip(img_np, 0, 1)
        img_np = (img_np * 255).astype(np.uint8)

        # blur icons with 30% probability
        img_np = blur_icons(img_np, yolo_data['annotations'], prob=0.3)

        img_pil = Image.fromarray(img_np)
        
        image_filename = f"minimap_{i:06d}.jpg"
        image_path = os.path.join(output_base_dir, image_filename)
        img_pil.save(image_path)
        
        # save YOLO format annotation file
        annotation_filename = f"minimap_{i:06d}.txt"
        annotation_path = os.path.join(output_base_dir, annotation_filename)
        with open(annotation_path, 'w') as f:
            for ann_str in yolo_data['annotations']:
                f.write(ann_str + '\n')
    except Exception as e:
        print(f"generating training sample {i} failed: {e}")

print(f"training set generated successfully, {minimap_samples} samples")


generating training set...


generating training set: 100%|██████████| 1/1 [00:00<00:00, 18.94it/s]

training set generated successfully, 1 samples
